In [2]:
#!pip install pytesseract transformers sentencepiece --quiet
!pip install pytesseract
import pytesseract
from PIL import Image,ImageOps,ImageFilter
from google.colab import files
import io
import re
import numpy as np



In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from transformers import pipeline
rephraser = pipeline("text2text-generation", model="vennify/t5-base-grammar-correction")#Model for grammer correction

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

Device set to use cpu


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report,accuracy_score
import joblib
import pandas as pd

In [7]:
train_df = pd.read_csv("/content/train.csv")
train_df = train_df[train_df['toxic'] != -1]

X = train_df['comment_text'].astype(str)
y = train_df[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

vectorizer = TfidfVectorizer(max_features=10000, stop_words='english')
X_train_vec = vectorizer.fit_transform(X_train)
X_val_vec = vectorizer.transform(X_val)

model = OneVsRestClassifier(LinearSVC())
model.fit(X_train_vec, y_train)

y_pred = model.predict(X_val_vec)
print("\n Classification Report:") #Classification Report
print(classification_report(y_val, y_pred, target_names=y.columns))

acc = accuracy_score(y_val, y_pred) #Accuracy
print("\nOverall Model Accuracy:",acc)

joblib.dump(model, 'multi_label_svm_model.pkl')
joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')


 Classification Report:
               precision    recall  f1-score   support

        toxic       0.86      0.68      0.76      3056
 severe_toxic       0.53      0.27      0.36       321
      obscene       0.89      0.70      0.78      1715
       threat       0.56      0.24      0.34        74
       insult       0.80      0.57      0.66      1614
identity_hate       0.73      0.28      0.40       294

    micro avg       0.84      0.62      0.71      7074
    macro avg       0.73      0.46      0.55      7074
 weighted avg       0.83      0.62      0.71      7074
  samples avg       0.06      0.06      0.06      7074


Overall Model Accuracy: 0.9196929343568855


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


['tfidf_vectorizer.pkl']

In [8]:
import cv2
def blurry(img, threshold=100.0):
    img_cv = np.array(img.convert('L')) #converting the image into grayscale (does't work with the colors)
    laplacian_var = cv2.Laplacian(img_cv, cv2.CV_64F).var() #Using 64 float/ Calculating varience to detect sharp edges
    return laplacian_var < threshold #if varience is less then 100 return

In [9]:
def preprocess(img): #Applying grayscale,Contrast and filter
    img=img.convert('L')
    img=ImageOps.autocontrast(img)
    if blurry(img):
        print("Image is blurry")
        img = img.filter(ImageFilter.UnsharpMask(radius=2, percent=150, threshold=3))
    else:
        print("Image is sharp enough")

    return img

def clean_ocr(text):
    text=text.encode('ascii',errors='ignore').decode()
    text=re.sub(r'[^A-Za-z0-9\s.,!?\'"/]+', ' ', text)#Regex use to avoid dummy values
    text=re.sub(r'\s+',' ',text) #Normalizing spaces
    return text.strip()

In [10]:
def confidence_score(img, confidence_threshold=70):#setting it to 70
    data = pytesseract.image_to_data(img, output_type=pytesseract.Output.DICT)# Returning 2 dictionaires for words and scores
    words = []
    for i in range(len(data['text'])):
        word = data['text'][i].strip()
        try:
          conf = int(data['conf'][i])
        except:
          conf=0
        if word and conf >= confidence_threshold:
            words.append(word)
    return ' '.join(words)


In [11]:
uploaded = files.upload()
print("Image is uploaded\n")

# Allowed image formats
allowed_images = ('.jpg', '.jpeg', '.png')

# Process each uploaded image from bytes into PIL readable
for file_name in uploaded.keys():
    if not file_name.lower().endswith(allowed_images):
        print(f"{file_name} is not an allowed image type")
        continue

    img = Image.open(io.BytesIO(uploaded[file_name]))
    img=preprocess(img)

    extracted_text = confidence_score(img)
    cleaned_text=clean_ocr(extracted_text)

    if cleaned_text.strip():
        enhanced_text = rephraser("fix grammar: " + cleaned_text, max_length=256)[0]['generated_text']
        model = joblib.load('multi_label_svm_model.pkl')
        vectorizer = joblib.load('tfidf_vectorizer.pkl')

        text_vec = vectorizer.transform([enhanced_text])
        prediction = model.predict(text_vec)[0]
        labels = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
        result = dict(zip(labels, prediction))

        print("\nEnhanced Text:\n", enhanced_text)
        print("\nPredicted Labels:")
        for label, value in result.items():
            print(f"{label}: {'Yes' if value == 1 else 'No'}")
    else:
        print(f"No readable text found in {file_name}")


Saving Screenshot 2025-06-23 101357.png to Screenshot 2025-06-23 101357.png
Image is uploaded

Image is sharp enough


Both `max_new_tokens` (=256) and `max_length`(=256) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



Enhanced Text:
 Hell you mother fucker!

Predicted Labels:
toxic: Yes
severe_toxic: No
obscene: Yes
threat: No
insult: Yes
identity_hate: No


In [12]:
!pip install flask-ngrok flask transformers pytesseract joblib scikit-learn opencv-python-headless --quiet

In [13]:
#For Flask
from flask import Flask, request, render_template_string
!pip install pyngrok
from pyngrok import ngrok

In [14]:
"""# Load the trained model, TF-IDF vectorizer, and grammar correction pipeline
model = joblib.load('multi_label_svm_model.pkl')
vectorizer = joblib.load('tfidf_vectorizer.pkl')
rephraser = pipeline("text2text-generation", model="vennify/t5-base-grammar-correction")
# Check if the image is blurry using the Laplacian variance method
def blurry(img, threshold=100.0):
    img_cv = np.array(img.convert('L'))
    laplacian_var = cv2.Laplacian(img_cv, cv2.CV_64F).var()
    return laplacian_var < threshold

# Preprocess the image: grayscale, enhance contrast, and sharpen if blurry
def preprocess(img):
    img = img.convert('L')
    img = ImageOps.autocontrast(img)
    if blurry(img):
        img = img.filter(ImageFilter.UnsharpMask(radius=2, percent=150, threshold=3))
    return img

# Extract words from OCR result with a minimum confidence score
def confidence_score(img, confidence_threshold=70):
    data = pytesseract.image_to_data(img, output_type=pytesseract.Output.DICT)
    words = []
    for i in range(len(data['text'])):
        word = data['text'][i].strip()
        try:
            conf = int(data['conf'][i])
        except:
            conf = 0
        if word and conf >= confidence_threshold:
            words.append(word)
    return ' '.join(words)

# Clean OCR text by removing non-ASCII characters and excess whitespace
def clean_ocr(text):
    text = text.encode('ascii', errors='ignore').decode()
    text = re.sub(r'[^A-Za-z0-9\s.,!?\'"/]+', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()"""


'# Load the trained model, TF-IDF vectorizer, and grammar correction pipeline\nmodel = joblib.load(\'multi_label_svm_model.pkl\')\nvectorizer = joblib.load(\'tfidf_vectorizer.pkl\')\nrephraser = pipeline("text2text-generation", model="vennify/t5-base-grammar-correction")\n# Check if the image is blurry using the Laplacian variance method\ndef blurry(img, threshold=100.0):\n    img_cv = np.array(img.convert(\'L\'))\n    laplacian_var = cv2.Laplacian(img_cv, cv2.CV_64F).var()\n    return laplacian_var < threshold\n\n# Preprocess the image: grayscale, enhance contrast, and sharpen if blurry\ndef preprocess(img):\n    img = img.convert(\'L\')\n    img = ImageOps.autocontrast(img)\n    if blurry(img):\n        img = img.filter(ImageFilter.UnsharpMask(radius=2, percent=150, threshold=3))\n    return img\n\n# Extract words from OCR result with a minimum confidence score\ndef confidence_score(img, confidence_threshold=70):\n    data = pytesseract.image_to_data(img, output_type=pytesseract.Outp

In [15]:
html='''
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <title>AI Text Extractor</title>
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css" rel="stylesheet">
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.5.0/css/all.min.css">
  <style>
    body {
      font-family: 'Segoe UI', sans-serif;
    }

    .container {
      max-width: 800px;
      margin-top: 40px;
    }

    .drop-area {
      border: 2px dashed;
      padding: 30px;
      text-align: center;
      border-radius: 10px;
      cursor: pointer;
      transition: border-color 0.3s;
    }

    .drop-area.dragover {
      border-color: #00d9ff !important;
    }

    .preview-img {
      max-width: 100%;
      max-height: 300px;
      margin-top: 15px;
      border-radius: 8px;
      border: 1px solid #ccc;
    }

    .result-text {
      white-space: pre-wrap;
      padding: 15px;
      border-radius: 8px;
      margin-bottom: 10px;
    }

    .card {
      border-radius: 12px;
    }

    .list-group-item {
      transition: background-color 0.3s, color 0.3s;
    }

    /* Static Progress Bar */
    #loaderWrapper {
      display: none;
      margin-top: 20px;
    }

    #loaderLabel {
      text-align: center;
      font-weight: 500;
      font-size: 0.95rem;
      margin-bottom: 8px;
    }

    #staticProgressBar {
      width: 0%;
      height: 16px;
      border-radius: 10px;
      background: linear-gradient(to right, #00c6ff, #0072ff);
      box-shadow: inset 0 0 4px rgba(0, 0, 0, 0.2);
      transition: width 0.2s ease;
    }

    /* Footer */
    .footer .social-icons a {
      color: #333;
      margin: 0 10px;
      font-size: 1.5rem;
      transition: color 0.3s, transform 0.2s;
      display: inline-block;
    }

    .footer .social-icons a:hover {
      color: #0d6efd;
      transform: scale(1.2);
    }

    .footer .footer-text {
      font-size: 0.95rem;
      color: #555;
    }

    @media (prefers-color-scheme: dark) {
      body {
        background-color: #121212;
        color: #eee;
      }

      .card {
        background-color: #1d1d1d;
        color: #ddd;
      }

      .drop-area {
        background-color: #1e1e1e;
        border-color: #555;
        color: #aaa;
      }

      .result-text {
        background-color: #1e1e1e;
        border: 1px solid #444;
        color: #cfcfcf;
      }

      .list-group-item {
        background-color: #1d1d1d;
        color: #ddd;
        border-color: #444;
      }

      .footer .social-icons a {
        color: #ccc;
      }

      .footer .social-icons a:hover {
        color: #66ccff;
      }

      .footer .footer-text {
        color: #aaa;
      }
    }
  </style>
</head>
<body>
<div class="container">
  <div class="text-center mb-4">
    <h2>AI Text Extractor</h2>
  </div>

  <div class="card p-4 mb-3">
    <form id="uploadForm" method="POST" enctype="multipart/form-data">
      <div class="drop-area" id="drop-area">
        Drag & Drop or Click to Upload Image (.jpg, .jpeg, .png)
        <input type="file" name="file" id="fileInput" accept=".jpg,.jpeg,.png" hidden required>
        <img id="preview" class="preview-img" style="display:none;">
      </div>

      <!-- Static Loader -->
      <div id="loaderWrapper">
        <div id="loaderLabel">Processing...</div>
        <div class="progress">
          <div id="staticProgressBar" class="progress-bar"></div>
        </div>
      </div>

      <div class="text-center mt-3">
        <button class="btn btn-primary" type="submit">Extract Text</button>
      </div>
    </form>
  </div>

  <div id="error" class="alert alert-danger d-none"></div>

  {% if result %}
  <div class="card p-4 mt-4">
    <h4>Enhanced Text:</h4>
    <div class="result-text" id="extractedText">{{ result['text'] }}</div>
    <div class="text-end mt-2">
      <button class="btn btn-secondary btn-sm" onclick="copyText()">Copy</button>
      <button class="btn btn-success btn-sm" onclick="downloadText()">Download</button>
    </div>

    <h4 class="mt-4">Predicted Labels:</h4>
    <ul class="list-group">
      {% for label, value in result['labels'].items() %}
      <li class="list-group-item d-flex justify-content-between align-items-center">
        {{ label.capitalize() }}
        <span class="badge bg-{{ 'success' if value == 1 else 'secondary' }}">
          {{ 'Yes' if value == 1 else 'No' }}
        </span>
      </li>
      {% endfor %}
    </ul>
  </div>
  {% endif %}
</div>

<footer class="footer mt-5 py-4 text-center bg-dark border-top">
  <div class="container">
    <div class="social-icons mb-3">
      <!-- Ali's Socials -->
      <a href="https://github.com/Ali-irtza" target="_blank" aria-label="Ali's GitHub" class="mx-2 text-white text-decoration-none">
        <i class="fab fa-github fa-lg"></i> Ali
      </a>
      <a href="https://linkedin.com/in/ali-irtza-a2827a27b" target="_blank" aria-label="Ali's LinkedIn" class="mx-2 text-info text-decoration-none">
        <i class="fab fa-linkedin fa-lg"></i>
      </a>

      <!-- Saad's Socials -->
      <a href="https://github.com/itzzsaadi" target="_blank" aria-label="Saad's GitHub" class="mx-2 text-white text-decoration-none">
        <i class="fab fa-github fa-lg"></i> Saad
      </a>
      <a href="https://linkedin.com/in/saad-naseer-b66ba617b" target="_blank" aria-label="Saad's LinkedIn" class="mx-2 text-info text-decoration-none">
        <i class="fab fa-linkedin fa-lg"></i>
      </a>
    </div>

    <div class="footer-text fw-semibold text-white">
      © 2025 Made by Ali Irtza & Saad Naseer
    </div>
    <div class="footer-text text-secondary small">
      <span>Ali Irtza – OCR & Model Training</span> |
      <span>Saad Naseer – Flask Backend & UI</span>
    </div>
  </div>
</footer>

<script>
  const dropArea = document.getElementById("drop-area");
  const fileInput = document.getElementById("fileInput");
  const preview = document.getElementById("preview");
  const errorDiv = document.getElementById("error");
  const form = document.getElementById("uploadForm");

  const loaderWrapper = document.getElementById("loaderWrapper");
  const staticProgressBar = document.getElementById("staticProgressBar");
  const loaderLabel = document.getElementById("loaderLabel");

  dropArea.addEventListener("click", () => fileInput.click());
  dropArea.addEventListener("dragover", e => {
    e.preventDefault();
    dropArea.classList.add("dragover");
  });
  dropArea.addEventListener("dragleave", () => dropArea.classList.remove("dragover"));
  dropArea.addEventListener("drop", e => {
    e.preventDefault();
    dropArea.classList.remove("dragover");
    const file = e.dataTransfer.files[0];
    if (file) {
      fileInput.files = e.dataTransfer.files;
      handleFile(file);
    }
  });

  fileInput.addEventListener("change", () => {
    const file = fileInput.files[0];
    handleFile(file);
  });

  function handleFile(file) {
    if (!file.type.startsWith('image/')) {
      showError("Only image files (.jpg, .jpeg, .png) are allowed.");
      fileInput.value = "";
      preview.style.display = "none";
      return;
    }
    errorDiv.classList.add("d-none");
    const reader = new FileReader();
    reader.onload = e => {
      preview.src = e.target.result;
      preview.style.display = "block";
    };
    reader.readAsDataURL(file);
  }

  function showError(msg) {
    errorDiv.classList.remove("d-none");
    errorDiv.innerText = msg;
  }

  form.addEventListener("submit", (e) => {
    e.preventDefault();
    loaderWrapper.style.display = "block";
    let percent = 0;
    const interval = setInterval(() => {
      percent += 2;
      staticProgressBar.style.width = percent + "%";
      if (percent >= 100) {
        clearInterval(interval);
        loaderLabel.innerText = "Complete!";
        setTimeout(() => {
          form.submit(); // finally submit the form
        }, 1000);
      }
    }, 120);
  });

  function copyText() {
    const text = document.getElementById("extractedText").innerText;
    navigator.clipboard.writeText(text);
  }

  function downloadText() {
    const text = document.getElementById("extractedText").innerText;
    const blob = new Blob([text], { type: "text/plain" });
    const link = document.createElement("a");
    link.download = "extracted_text.txt";
    link.href = URL.createObjectURL(blob);
    link.click();
  }
</script>
</body>
</html>
'''

In [16]:

app = Flask(__name__)
@app.route('/', methods=['GET', 'POST'])
def index():
    result = None
    if request.method == 'POST':
        file = request.files['file']

        if file:
            img = Image.open(file.stream)
            img = preprocess(img)
            extracted_text = confidence_score(img)
            cleaned_text = clean_ocr(extracted_text)

            if cleaned_text.strip():
                enhanced_text = rephraser("fix grammar: " + cleaned_text, max_length=256)[0]['generated_text']

                text_vec = vectorizer.transform([enhanced_text])
                prediction = model.predict(text_vec)[0]

                labels = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
                result = {
                    "text": enhanced_text,
                    "labels": dict(zip(labels, prediction))
                }

    return render_template_string(html, result=result)


In [17]:
#Auth token to run application on a live server (Shareable)
from pyngrok import conf
conf.get_default().auth_token = "2dQPg7ECnBMgC2NBaqvV5C6Dwe0_3Kn1MojdBRGgD7Z85s1T1"

# Run the app with pyngrok
port = 5000
public_url = ngrok.connect(port)
print("🚀 Open this URL in a new tab:", public_url)

app.run(port=port)


🚀 Open this URL in a new tab: NgrokTunnel: "https://70ae-34-125-124-84.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [24/Jun/2025 18:40:24] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Jun/2025 18:40:26] "GET /favicon.ico HTTP/1.1" 404 -


Image is sharp enough


Both `max_new_tokens` (=256) and `max_length`(=256) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
INFO:werkzeug:127.0.0.1 - - [24/Jun/2025 18:40:48] "POST / HTTP/1.1" 200 -
